In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
#import datetime as dt
from datetime import datetime as dt

#logger class
class App_Logger:
    def __init__(self):
        pass

    def log(self, file_object, log_message):
        self.now = dt.now()
        self.date = self.now.date()
        self.current_time = self.now.strftime("%H:%M:%S")
        file_object.write(
            str(self.date) + "/" + str(self.current_time) + "\t\t" + log_message +"\n")

#Perprocessing Class
class data_preprocessing:
        def __init__(self, file_object, logger_object):
            self.file_object = file_object
            self.logger_object = logger_object
        def dropUnnecessaryColumns(self,data,columnNameList):
            data = data.drop(columnNameList,axis=1)
            return data
        def dropna(self,data):
            data=data.dropna()
            return data
        def replaceInvalidValuesWithNull(self,data):

            for column in data.columns:
                count = data[column][data[column] == '?'].count()
                if count != 0:
                    data[column] = data[column].replace('?', np.nan)
            return data

        def is_null_present(self,data):
                    
            self.logger_object.log(self.file_object, 'Entered the is_null_present method of the Preprocessor class')
            self.null_present = False
            try:
                self.null_counts=data.isna().sum() # check for the count of null values per column
                for i in self.null_counts:
                    if i>0:
                        self.null_present=True
                        break
                if(self.null_present): # write the logs to see which columns have null values
                    dataframe_with_null = pd.DataFrame()
                    dataframe_with_null['columns'] = data.columns
                    dataframe_with_null['missing values count'] = np.asarray(data.isna().sum())
                    dataframe_with_null.to_csv('preprocessing_data/null_values.csv') # storing the null column information to file
                self.logger_object.log(self.file_object,'Finding missing values is a success.Data written to the null values file. Exited the is_null_present method of the Preprocessor class')
                return self.null_present
            except Exception as e:
                self.logger_object.log(self.file_object,'Exception occured in is_null_present method of the Preprocessor class. Exception message:  ' + str(e))
                self.logger_object.log(self.file_object,'Finding missing values failed. Exited the is_null_present method of the Preprocessor class')
                raise Exception()
        def cal_age(self,data):
            data['Age'] = 2015 - data.Year_Birth
        def month_enroll(self,data):
            data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])
            data['Month_Customer'] = 12.0 * (2015 - data.Dt_Customer.dt.year ) + (1 - data.Dt_Customer.dt.month)
        def cal_expenses(self,data):
            data['Expenses'] =  data.MntWines + data.MntFruits + data.MntMeatProducts + data.MntFishProducts + data.MntSweetProducts + data.MntGoldProds
        def age_group(self,data):
            data.loc[(data['Age'] >= 13) & (data['Age'] <= 19), 'AgeGroup'] = 'Teen'
            data.loc[(data['Age'] >= 20) & (data['Age']<= 39), 'AgeGroup'] = 'Adult'
            data.loc[(data['Age'] >= 40) & (data['Age'] <= 59), 'AgeGroup'] = 'Middle Age Adult'
            data.loc[(data['Age'] > 60), 'AgeGroup'] = 'Senior Adult'
        def count_children(self,data):
            data['Children'] = data['Kidhome'] + data['Teenhome']
        def encodecategorialvalues(self,data):
            data.Marital_Status = data.Marital_Status.replace({'Together': 'Partner',
                                               'Married': 'Partner',
                                                'Divorced': 'Single',
                                                'Widow': 'Single', 
                                                'Alone': 'Single',
                                                'Absurd': 'Single',
                                                'YOLO': 'Single'})
